In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

df = pd.read_csv('../data/markets/BTCUSDT_1d.csv')
# first futures data in the API was 6/25/23, second month futs come online in the future but NaN until then.
df = df[df["Timestamp"] >= "2023-06-25"]

# make unit for annualized funding match others
df["funding_annualized"] *= 100
df.head()

In [ ]:
# calculate basis and fut apr/apy for prompt
df.rename(columns={"prompt_days_till_expiry": "prompt_dte"}, inplace=True)
df["prompt_basis"] = df["prompt_close"] - df["spot_close"]
df["prompt_apr"] = (df["prompt_basis"] / df["spot_close"]) * (365 / df["prompt_dte"]) * 100
df["prompt_apy"] = (np.exp(df["prompt_apr"] / 100) - 1) * 100

print(df[["Timestamp", "prompt_contract", "prompt_dte",
          "spot_close", "prompt_close", 
          "prompt_basis", "prompt_apr", "prompt_apy"]].head())

In [ ]:
# repeat for second fut
# bumping date to prevent NaN in head
df = df[df["Timestamp"] >= "2023-09-29"]

# calculate basis and fut apr/apy for prompt
df.rename(columns={"next_days_till_expiry": "next_dte"}, inplace=True)
df["next_basis"] = df["next_close"] - df["spot_close"]
df["next_apr"] = (df["next_basis"] / df["spot_close"]) * (365 / df["next_dte"]) * 100
df["next_apy"] = (np.exp(df["prompt_apr"] / 100) - 1) * 100

print(df[["Timestamp", "next_contract", "next_dte",
          "spot_close", "next_close", 
          "next_basis", "next_apr", "next_apy"]].head())

In [ ]:
# get the forward rate
df["forward_rate"] = ((df["next_close"] / df["prompt_close"]) - 1) * (365 / (df["next_dte"] - df["prompt_dte"])) * 100

# include binance interest_rate
df["binance_rate"] = 0.0001 * 3 * 365 * 100

print(df[["Timestamp", "prompt_apr", "next_apr", "forward_rate", "funding_annualized", "binance_rate"]].head())

In [ ]:
# Set up the figure
plt.figure(figsize=(12, 6))

# Plot all five rates using Seaborn
sns.lineplot(x=df["Timestamp"], y=df["prompt_apr"], color="blue", label="Prompt APR")
sns.lineplot(x=df["Timestamp"], y=df["next_apr"], color="red", label="Next APR")
sns.lineplot(x=df["Timestamp"], y=df["forward_rate"], color="green", label="Forward Rate")
sns.lineplot(x=df["Timestamp"], y=df["funding_annualized"], color="purple", label="Funding Annualized")
sns.lineplot(x=df["Timestamp"], y=df["binance_rate"], color="orange", linestyle="dashed", label="Binance Rate")

# Labels and title
plt.xlabel("Date")
plt.ylabel("Annualized Percentage Rate (APR)")
plt.title("Comparison of APR, Forward Rate, and Binance Rate")
plt.legend()

# Clean up x-axis labels (reduce number of labels)
plt.xticks(rotation=90)
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Show label every month

# Remove vertical grid lines but keep horizontal ones
plt.grid(axis="y", linestyle="--", alpha=0.6)  # Dashed horizontal grid only

# Show the plot
plt.show()

In [ ]:
# Subtract Binance rate from the other curves
df["prompt_apr"] -= df["binance_rate"]
df["next_apr"] -= df["binance_rate"]
df["forward_rate"] -= df["binance_rate"]
df["funding_annualized"] -= df["binance_rate"]

# Set up the figure
plt.figure(figsize=(12, 6))

# Plot adjusted rates using Seaborn
sns.lineplot(x=df["Timestamp"], y=df["prompt_apr"], color="blue", label="Adjusted Prompt APR")
sns.lineplot(x=df["Timestamp"], y=df["next_apr"], color="red", label="Adjusted Next APR")
sns.lineplot(x=df["Timestamp"], y=df["forward_rate"], color="green", label="Adjusted Forward Rate")
sns.lineplot(x=df["Timestamp"], y=df["funding_annualized"], color="purple", label="Adjusted Funding Annualized")

# Labels and title
plt.xlabel("Date")
plt.ylabel("Annualized Percentage Rate (APR) Adjusted for Binance Rate")
plt.title("Comparison of APR, Forward Rate, and Funding Rate (Adjusted)")
plt.legend()

# Clean up x-axis labels (reduce number of labels)
plt.xticks(rotation=90)
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Show label every month

# Remove vertical grid lines but keep horizontal ones
plt.grid(axis="y", linestyle="--", alpha=0.6)  # Dashed horizontal grid only

# Show the plot
plt.show()


In [ ]:
# Ensure necessary columns exist before plotting
if "funding_annualized" in df.columns and "prompt_apr" in df.columns:
    # Scatter Plot
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=df["funding_annualized"], y=df["prompt_apr"], color="blue", alpha=0.6)
    
    # Labels and title
    plt.xlabel("Funding Annualized (%)")
    plt.ylabel("Prompt APR (%)")
    plt.title("Scatter Plot of Funding Annualized vs. Prompt APR")
    plt.grid(True)
    
    # Show the plot
    plt.show()

# Compute funding condition statistics
funding_negative_count = (df["funding_annualized"] < -0.001).sum()
funding_negative_sum = df.loc[df["funding_annualized"] < -0.001, "funding_annualized"].sum()

funding_zero_count = ((df["funding_annualized"] >= -0.001) & (df["funding_annualized"] <= 0.001)).sum()
funding_zero_sum = df.loc[(df["funding_annualized"] >= -0.001) & (df["funding_annualized"] <= 0.001), "funding_annualized"].sum()

funding_positive_count = (df["funding_annualized"] > 0.001).sum()
funding_positive_sum = df.loc[df["funding_annualized"] > 0.001, "funding_annualized"].sum()

# Create a summary DataFrame
funding_stats = pd.DataFrame({
    "Condition": ["Funding < -0.001", "Funding ≈ 0 (±0.001)", "Funding > 0.001"],
    "Count": [funding_negative_count, funding_zero_count, funding_positive_count],
    "Sum": [funding_negative_sum, funding_zero_sum, funding_positive_sum]
})

# Display statistics
print(funding_stats)

In [ ]:

# Set up the figure
plt.figure(figsize=(12, 6))

# Histogram and KDE plot
sns.histplot(df["prompt_apr"], bins=100, kde=True, color="blue", alpha=0.6)

# Labels and title
plt.xlabel("Annualized Percentage Rate (APR)")
plt.ylabel("Frequency")
plt.title("Distribution of Prompt APR (Histogram & KDE)")
plt.grid(True)

# Show the plot
plt.show()
